<a href="https://colab.research.google.com/github/yashveersinghsohi/machine_hack_competitions/blob/baseline_model/machine_hack_competitions/Data_Science_Student_Championship/Baseline_Model/baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Git Clone

In [3]:
# !git clone -b baseline_model https://github.com/yashveersinghsohi/machine_hack_competitions.git

# Imports

In [35]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

# Datasets

In [36]:
root_dir = '/content/machine_hack_competitions/Data_Science_Student_Championship/Features/'
X_train = pd.read_csv(root_dir+'X_train.csv')
y_train = pd.read_csv(root_dir+'y_train.csv')

X_val = pd.read_csv(root_dir+'X_val.csv')
y_val = pd.read_csv(root_dir+'y_val.csv')

X_test = pd.read_csv(root_dir+'X_test.csv')
y_test = pd.read_csv(root_dir+'y_test.csv')

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((167738, 20), (167738, 1), (41935, 20), (41935, 1), (89861, 20), (89861, 1))

# Baseline XGBoost Regression

In [37]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train.to_numpy().reshape(-1,))

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [38]:
def calculate_metrics(datasets, model):
  X, y = datasets
  mae = mean_absolute_error(y_true=y, y_pred=model.predict(X)).round(4)
  mse = mean_squared_error(y_true=y, y_pred=model.predict(X)).round(4)
  rmse = np.sqrt(mse).round(4)
  # msle = mean_squared_log_error(y_true=y, y_pred=model.predict(X)).round(4)
  # rmsle = np.sqrt(msle).round(4)
  return {'mae': mae, 'mse': mse, 'rmse': rmse}

**Train Metrics**

In [39]:
datasets = X_train, y_train.to_numpy().reshape(-1,)
calculate_metrics(datasets=datasets, model=xgb)

{'mae': 5.2071, 'mse': 468.5473, 'rmse': 21.646}

**Val Metrics**

In [40]:
datasets = X_val, y_val.to_numpy().reshape(-1,)
calculate_metrics(datasets=datasets, model=xgb)

{'mae': 6.7944, 'mse': 1605.339, 'rmse': 40.0667}

**Test Metrics**

In [41]:
datasets = X_test, y_test.to_numpy().reshape(-1,)
calculate_metrics(datasets=datasets, model=xgb)

{'mae': 99.2661, 'mse': 16143.1762, 'rmse': 127.0558}

# Submission

In [47]:
def create_submission(
    model, X, 
    submission_path='/content/machine_hack_competitions/Data_Science_Student_Championship/Competition_Files/submission.csv'
  ):
  X_copy = X.copy()
  X_copy['fare_pred'] = model.predict(X)
  submission_df = pd.read_csv(submission_path)
  submission_df['total_fare'] = X_copy['tip'] + X_copy['miscellaneous_fees'] + X_copy['fare_pred']
  submission_df['total_fare'] = np.where(submission_df['total_fare'] < 0, 0, submission_df['total_fare'])
  print(submission_df.head())
  submission_df.to_csv('submission.csv', index=False)

In [48]:
create_submission(model=xgb, X=X_test)

   total_fare
0  111.457520
1   61.831718
2  117.722549
3   90.581764
4  114.545013
